In [23]:
import os
import glob
import pandas as pd
import itertools

import Bio
from Bio import Entrez, SeqIO
from Bio import AlignIO

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
path_to_bam_file = '/Users/unnost/workwork/prespacers_task/probnaya/Results_fragseq/Oct14_large_uni_mapped.csv'
df_intermed_path = ''
df_align = ''

#path_to_bam_file = '/Users/unnost/workwork/prespacers_task/Results_fragseq/Oct14_S10_R1_001_R2_kd403-real_uni_mapped.csv'
df_intermed_path = '/Users/unnost/workwork/prespacers_task/probnaya/Results_fragseq/Oct14_large_cut.csv'
df_align = '/Users/unnost/workwork/prespacers_task/probnaya/Results_fragseq/Oct14_large_align_df.csv'

In [38]:
bbbb = pd.read_csv(path_to_bam_file,  sep =' ') #, header=None)
bbbb.shape[0]

7074243

In [39]:
bbbb.head()

,qname,flag,rname,strand,pos,qwidth,mapq,cigar,mrnm,mpos,isize,seq,qual
1,A01492:11:HNH7TDMXX:1:1101:5710:1016,83,KD403,-,4436683,95,60,95M,KD403,4436683,-95,GTAGTATTCGTTATCCGATCCAGGAATGGTCACTTTACCGGTCAGG...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
2,A01492:11:HNH7TDMXX:1:1101:5710:1016,163,KD403,+,4436683,95,60,95M,KD403,4436683,95,GTAGTATTCGTTATCCGATCCAGGAATGGTCACTTTACCGGTCAGG...,FF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
3,A01492:11:HNH7TDMXX:1:1101:13630:1016,83,KD403,-,3139284,59,60,59M,KD403,3139284,-59,GACGGTTCTCAACGTTATGCCAGAGCGATTTTTCGGCGTCGTATTG...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
4,A01492:11:HNH7TDMXX:1:1101:13630:1016,163,KD403,+,3139284,59,60,59M,KD403,3139284,59,GACGGTTCTCAACGTTATGCCAGAGCGATTTTTCGGCGTCGTATTG...,FFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFF...
5,A01492:11:HNH7TDMXX:1:1101:17616:1031,99,KD403,+,2198430,96,60,96M,KD403,2198433,96,GATTATCTCCCTGACCTGGTGGTTGCCCAGGAGGAGGGCCGGAAAT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...


In [52]:
#here i could check whether we deal with several references or only one
#bbbb.drop_duplicates(subset = 'DNA', keep = 'first', inplace = True)
#bbbb.head(10)

In [40]:
bbbb.rename(columns = {'qname' : 'ID', 'mrnm' : 'DNA', 'mpos' : 'left', 'isize' : 'width'}, inplace = True)
bbbb = bbbb.drop(columns = ['pos','qwidth', 'mapq', 'seq','qual'], axis = 1)
#leave only one read from pair
bbbb.drop_duplicates(subset = 'ID', keep = 'first', inplace = True)

In [41]:
bbbb.shape[0]

3536187

In [42]:
bbbb.head()

,ID,flag,rname,strand,cigar,DNA,left,width
1,A01492:11:HNH7TDMXX:1:1101:5710:1016,83,KD403,-,95M,KD403,4436683,-95
3,A01492:11:HNH7TDMXX:1:1101:13630:1016,83,KD403,-,59M,KD403,3139284,-59
5,A01492:11:HNH7TDMXX:1:1101:17616:1031,99,KD403,+,96M,KD403,2198433,96
7,A01492:11:HNH7TDMXX:1:1101:7247:1047,99,KD403,+,54M,KD403,2839958,54
9,A01492:11:HNH7TDMXX:1:1101:1398:1063,83,KD403,-,57M,KD403,895088,-57


In [44]:
bbbb.to_csv(df_intermed_path, index=False)

In [53]:
bbbb = pd.read_csv(df_intermed_path)#,  sep =' ') #, header=None)
bbbb.head()

,ID,flag,rname,strand,cigar,DNA,left,width
0,A01492:11:HNH7TDMXX:1:1101:5710:1016,83,KD403,-,95M,KD403,4436683,-95
1,A01492:11:HNH7TDMXX:1:1101:13630:1016,83,KD403,-,59M,KD403,3139284,-59
2,A01492:11:HNH7TDMXX:1:1101:17616:1031,99,KD403,+,96M,KD403,2198433,96
3,A01492:11:HNH7TDMXX:1:1101:7247:1047,99,KD403,+,54M,KD403,2839958,54
4,A01492:11:HNH7TDMXX:1:1101:1398:1063,83,KD403,-,57M,KD403,895088,-57


In [54]:
bbbb = bbbb.query("flag in [83, 99, 163, 147]")
bbbb = bbbb.drop(columns = ['flag'], axis = 1)
bbbb = bbbb[~bbbb.cigar.str.contains("D")]
bbbb = bbbb[~bbbb.cigar.str.contains("I")]

In [55]:
bbbb.shape[0]

3524728

In [56]:
def plus_minus(s):
    if s == "-":
        return "rev"
    elif s == "+":
        return "dir"
    
def module_val(s):
    return abs(s)

In [57]:
bbbb['strand'] = bbbb['strand'].apply(plus_minus)
bbbb['width'] = bbbb['width'].apply(module_val)
bbbb['right'] = bbbb['left'] + bbbb['width'] - 1

bbbb.head()

,ID,rname,strand,cigar,DNA,left,width,right
0,A01492:11:HNH7TDMXX:1:1101:5710:1016,KD403,rev,95M,KD403,4436683,95,4436777
1,A01492:11:HNH7TDMXX:1:1101:13630:1016,KD403,rev,59M,KD403,3139284,59,3139342
2,A01492:11:HNH7TDMXX:1:1101:17616:1031,KD403,dir,96M,KD403,2198433,96,2198528
3,A01492:11:HNH7TDMXX:1:1101:7247:1047,KD403,dir,54M,KD403,2839958,54,2840011
4,A01492:11:HNH7TDMXX:1:1101:1398:1063,KD403,rev,57M,KD403,895088,57,895144


In [58]:
bbbb = bbbb.drop(columns = ['rname', 'cigar'], axis = 1)
bbbb.rename(columns = {'width' : 'isize'}, inplace = True)
bbbb['width'] = bbbb['isize']
bbbb = bbbb.drop(columns = ['isize'], axis = 1)

bbbb.head()

,ID,strand,DNA,left,right,width
0,A01492:11:HNH7TDMXX:1:1101:5710:1016,rev,KD403,4436683,4436777,95
1,A01492:11:HNH7TDMXX:1:1101:13630:1016,rev,KD403,3139284,3139342,59
2,A01492:11:HNH7TDMXX:1:1101:17616:1031,dir,KD403,2198433,2198528,96
3,A01492:11:HNH7TDMXX:1:1101:7247:1047,dir,KD403,2839958,2840011,54
4,A01492:11:HNH7TDMXX:1:1101:1398:1063,rev,KD403,895088,895144,57


In [59]:
#df_align = '/Users/unnost/workwork/prespacers_task/Results_fragseq/Oct14_align_df.csv'
bbbb.to_csv(df_align, index=False)